In [1]:
import numpy as np
import pandas as pd
import pandana as pdna
import geopandas as gp
import matplotlib.pyplot as plt

In [2]:
homedata_file_path = '/home/data/'
dd = '/home/data/spring_2019/base/'
shapefiles_path = '/home/simon/spatial-data/'
model_outputs_path = '/home/data/spring_2019/outputs/'


In [3]:
def get_data():
    '''gets all relevant tables from the HDF5 and saves them to a dictionary that is indexed by 1) year and 
    2) table name. Closes the store. The function is assigned to the global variable data_dict'''
    data_dict = {}
    hdfstore = pd.HDFStore(model_outputs_path+'model_data_output.h5', mode='r') #sometimes doesnt work with a relative file path
    for year in [2010, 2015, 2025]:
        persons = hdfstore["/{}/persons".format(year)].head(100000)
        hh = hdfstore["/{}/households".format(year)].head(100000)
        buildings = hdfstore["/{}/buildings".format(year)].head(100000)
        parcels = hdfstore["/{}/parcels".format(year)].head(100000)
        jobs = hdfstore["/{}/jobs".format(year)].head(100000)
        data_dict[year] = {"hh": hh, "buildings": buildings, "parcels": parcels, "jobs": jobs, "persons": persons}
    print("closing store")
    hdfstore.close()
    return data_dict

data_dict = get_data()

closing store


In [4]:
def get_merged_data(year):
    '''merges the household and buildings and parcels and persons tables together in an inner merge, 
    resulting in a dataframe with every person, their household information and their geographic information
    like block group id and parcel id, and node id'''
        
    print("merging buildings to parcels")
    try:
        buildings = data_dict[year]["buildings"].reset_index()[["building_id", "parcel_id"]]
        parcels = data_dict[year]["parcels"].reset_index()[["parcel_id", "county_id", "node_id"]]
        bldg_parc = pd.merge(buildings, parcels, on="parcel_id")
        hh = data_dict[year]["hh"]
        persons = data_dict[year]["persons"]
    except Exception as e:
        print(e)
#         import pdb; pdb.set_trace()
    
    print("merging households to buildings and parcels table")
    
    hh_build = pd.merge(hh, bldg_parc, on="building_id")
    
    print("merging households to buildings")

    hh_build.index.name = "household_id"
    
    hh_build = hh_build.reset_index()
    print("merging persons to households and buildings")
    persons = data_dict[year]["persons"]
    persons.index.name = "person_id"
    
    buildings_persons = pd.merge(persons, hh_build, on='household_id')
    print("hh build table has ", len(hh_build), "rows")
    
    buildings_persons["block_group_id"] = buildings_persons["block_group_id"].apply(lambda x: "0"+x)
    buildings_persons["census_tract"] = buildings_persons["block_group_id"].apply(lambda x: x[:-1])
    return buildings_persons

In [5]:
def get_job_locations(year):
    '''get the location of the jobs by merging the jobs table with the buildings table on building_id and that
    table with the parcels table on parcel_id, resulting in each job being assigned to a node id to be used
    in the pandana network'''
    
    buildings = data_dict[year]['buildings'].reset_index()[['building_id', 'parcel_id']]
    
    jobs = data_dict[year]['jobs'].reset_index()[['occupation_id', 'building_id', 'sector_id']]
    
    parcels = data_dict[year]['parcels'][['parcel_id_local', 'block_id', 'county_id', 'y', 'x', 'node_id']].reset_index()
    
    print("merging buildings to jobs")
    jobs_buildings = pd.merge(buildings, jobs, on='building_id')
    
    print("merging buildings with jobs to parcels")
    jobs_merge = pd.merge(jobs_buildings, parcels, on='parcel_id')
    jobs_merge['census_tract'] = jobs_merge['block_id'].apply(lambda x: x[:-4])
    jobs_merge['block_group'] = jobs_merge['block_id'].apply(lambda x: x[:-3])

    return jobs_merge


In [14]:
def get_workers_by_race(year):
    '''Get the number of workers for each race by node, for a given year. Returns a dataframe
    with a column for each race and the number of workers assigned to that node by race. The node assignments
    are obtained from the person-> parcel match in get_merged_data()'''
    
    node_df_lol = []
    node_id_list = []
    df = get_merged_data(year)
    for node_id, node_df in df.groupby("node_id"):
        nhwhite_workers = sum((node_df["race_id"] == 1) & (node_df["hispanic"] == 1) & (node_df["worker"] == 1))
        nhblack_workers = sum((node_df["race_id"] == 2) & (node_df["hispanic"] == 1) & (node_df["worker"] == 1))
        nhasian_workers = sum((node_df["race_id"] == 6) & (node_df["hispanic"] == 1) & (node_df["worker"] == 1))
        hispanic_workers = sum((node_df["hispanic"] != 1) & (node_df["worker"] == 1))
        non_workers_any = sum(node_df["worker"] == 0)
        block = node_df['block_group_id'].iloc[0] # i think this is ok for now because they should all be the same
        node_id_list.append(node_id)

        row = [nhwhite_workers, nhblack_workers, nhasian_workers, hispanic_workers, non_workers_any, block]
        node_df_lol.append(row)
        
    workers_by_node = pd.DataFrame(node_df_lol, columns=["nhwhite_workers", "nhblack_workers","nhasian_workers",
            "hispanic_workers", "non_workers_any", "block_group"], index=node_id_list)
    workers_by_node['census_tract'] = workers_by_node['block_group'].apply(lambda x: x[:-1])
    
    return workers_by_node


In [6]:
# get_workers_by_race(2025)

In [11]:
beam_network_links = pd.read_csv('beam-network-links.csv')
beam_network_nodes = pd.read_csv('beam-network-nodes.csv')

net=pdna.Network(beam_network_nodes.x, beam_network_nodes.y, beam_network_links["from"], beam_network_links["to"],
                 beam_network_links[["travelTime"]])
net.precompute(10000)
del beam_network_links
del beam_network_nodes

/home/amelia/enter/envs/template-env/lib/python3.6/site-packages/pandana/network.py:132: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  nodes_df.as_matrix().astype('float32'),
/home/amelia/enter/envs/template-env/lib/python3.6/site-packages/pandana/network.py:133: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  edges.as_matrix().astype('int32'),
/home/amelia/enter/envs/template-env/lib/python3.6/site-packages/pandana/network.py:134: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  edges_df[edge_weights.columns].transpose()


In [44]:
def get_job_accessibilities(net, year, dist):
    '''returns the number of jobs within a radius of dist of each node'''
    df = get_job_locations(year)
    workers_df = get_workers_by_race(year)
    node_ids = df["node_id"]
    n = net.set(node_ids)
    s = net.aggregate(dist, type="sum", decay="linear")
    access = pd.DataFrame(s)
    access = access.rename({0:"jobs"}, inplace=False, axis=1)
    access = pd.merge(workers_df, access, right_index=True, left_index=True)
    return access

In [45]:
def group_access_by_geo(job_access_df):
    people_per_geo = sum(job_access_df["nhwhite_workers"] + job_access_df["nhblack_workers"]+job_access_df["nhasian_workers"]+job_access_df["hispanic_workers"])+1e-6
    geo_access_nhw = sum(job_access_df["nhwhite_workers"]*job_access_df["jobs"])/people_per_geo
    geo_access_nhb = sum(job_access_df["nhblack_workers"]*job_access_df["jobs"])/people_per_geo
    geo_access_nha = sum(job_access_df["nhasian_workers"]*job_access_df["jobs"])/people_per_geo
    geo_access_h = sum(job_access_df["hispanic_workers"]*job_access_df["jobs"])/people_per_geo
    jobs = sum(job_access_df["jobs"])
    return pd.Series([geo_access_nhw, geo_access_nhb, geo_access_nha, geo_access_h, jobs])



$$\text{weighted accessibility} = E[\text{# of accessible jobs}|\text{race R}] =$$

$$\sum_{i=1}^n P(\text{person of race R and person assigned to node}_i) \times \text{# jobs assigned to node}_i =$$

$$\sum_{i=1}^n P(\text{person of race R | person assigned to node}_i) \times P(\text{person assigned to node}_i) \times \text{# jobs assigned to node}_i =$$

$$\sum_{i=1}^n \frac{\text{# persons of race R assigned to node}_i}{\text{# persons assigned to node}_i} \times \frac{\text{# persons assigned to node}_i}{\text{# of people in blk group}} \times (\text{# of jobs assigned to node}_i) =$$

$$\sum_{i=1}^n \frac{\text{# persons of race R assigned to node}_i}{\text{# of people in blk group}} \times (\text{# of jobs assigned to node}_i)$$

where $i$ is a node in the block group/other geography.

In [46]:
def accessibilities_by_race(job_accessibilities_df, year, dist, net, geography): 
    '''This is the the essence of the computation being performed to get the race-based accessibilities. This function
    takes in the year, buffer distance, pandana network (should be precomputed) and desired geography (right now this
    notebook has shapefiles set up for census tract ("census_tract") and block group ("block_group"). Returns a table indexed by 
    the id of the geography, and a column for each weighted accessibility by race.'''    

    if geography == 'node':
        access_by_race_df = pd.DataFrame()
        people_per_geo = job_accessibilities_df["nhwhite_workers"] + job_accessibilities_df["nhblack_workers"]+job_accessibilities_df["nhasian_workers"]+job_accessibilities_df["hispanic_workers"]+1e-6
        node_access_nhw = job_accessibilities_df["nhwhite_workers"]*job_accessibilities_df["jobs"]/people_per_geo
        node_access_nhb = job_accessibilities_df["nhblack_workers"]*job_accessibilities_df["jobs"]/people_per_geo
        node_access_nha = job_accessibilities_df["nhasian_workers"]*job_accessibilities_df["jobs"]/people_per_geo
        node_access_h = job_accessibilities_df["hispanic_workers"]*job_accessibilities_df["jobs"]/people_per_geo
        
        access_by_race_df["node_access_nhw"] = node_access_nhw
        access_by_race_df["node_access_nhb"] = node_access_nhb
        access_by_race_df["node_access_nha"] = node_access_nha
        access_by_race_df["node_access_h"] = node_access_h
        display(access_by_race_df)
        return access_by_race_df
    else:
        access_geo = job_accessibilities_df.groupby(geography).apply(group_access_by_geo)
        access_geo.columns = ['geo_access_nhw', 'geo_access_nhb', 'geo_access_nha', 'geo_access_h', 'jobs']
        return access_geo
        
        
        
        

In [47]:
bay_county_codes = ['001', '013', '097', '095', '081', '085', '075', '041', '055'] 
block_groups = gp.read_file(shapefiles_path+'tl_2018_06_bg/tl_2018_06_bg.shp')
bay_bg = block_groups[(block_groups["COUNTYFP"].isin(bay_county_codes)) & (block_groups["ALAND"]> 100)]
del block_groups

tracts = gp.read_file(shapefiles_path+'tl_2018_06_tract/tl_2018_06_tract.shp')
bay_tracts = tracts[(tracts["COUNTYFP"].isin(bay_county_codes)) & (tracts["ALAND"]> 100)]
del tracts

In [67]:
def get_access_by_race_geo(year, dist, net, geographies):
    '''merge the relevant shapefiles with the race weighted accessibilities tables to allow mapping. geographies is a list of 
    desired geographies'''
    access_by_geos = {}
    job_accessibilities_df = get_job_accessibilities(net, year, dist)

    
    for geography in geographies:
        job_access_by_race_df = accessibilities_by_race(job_accessibilities_df, year, dist, net, geography)
        if (geography == 'node'):
            access_by_geo = job_access_by_race_df
        elif (geography == 'census_tract'):
            access_by_geo = pd.merge(bay_tracts, job_access_by_race_df, right_index=True, left_on='GEOID')
        elif (geography == 'block_group'):
            access_by_geo = pd.merge(bay_bg, job_access_by_race_df, right_index=True, left_on='GEOID')
        
        access_by_geo.index = job_access_by_race_df.index

        access_by_geos[geography] = access_by_geo
    return access_by_geos


In [69]:
access_10000_2025 = get_access_by_race_geo(2025, 10000, net, ["census_tract", "block_group", "node"])
for geo, df in access_10000_2025.items():
    df.to_csv("indicators_output/access_10000_2025_{0}.csv".format(geo))
del access_10000_2025
access_10000_2015 = get_access_by_race_geo(2015, 10000, net, ["census_tract", "block_group", "node"])
for geo, df in access_10000_2015.items():
    df.to_csv("indicators_output/access_10000_2015_{0}.csv".format(geo))
del access_10000_2015
access_10000_2010 = get_access_by_race_geo(2010, 10000, net, ["census_tract", "block_group", "node"])
for geo, df in access_10000_2010.items():
    df.to_csv("indicators_output/access_10000_2010_{0}.csv".format(geo))
del access_10000_2010

merging buildings to jobs
merging buildings with jobs to parcels
merging buildings to parcels
merging households to buildings and parcels table
merging households to buildings
merging persons to households and buildings
hh build table has  423 rows
Removed 571 rows because they contain missing values


,node_access_nhw,node_access_nhb,node_access_nha,node_access_h
5030,88.478313,0.0,0.000000,0.000000
12239,0.000000,0.0,0.000000,91.519951
14466,0.000000,0.0,78.168089,0.000000
15402,94.121877,0.0,0.000000,0.000000
15704,0.000000,0.0,0.000000,0.000000
16027,0.000000,0.0,0.000000,91.691460
16544,0.000000,0.0,62.492751,0.000000
23566,0.000000,0.0,0.000000,0.000000
29303,92.250785,0.0,0.000000,0.000000
30641,0.000000,0.0,84.899604,0.000000


merging buildings to jobs
merging buildings with jobs to parcels
merging buildings to parcels
merging households to buildings and parcels table
merging households to buildings
merging persons to households and buildings
hh build table has  421 rows
Removed 537 rows because they contain missing values


,node_access_nhw,node_access_nhb,node_access_nha,node_access_h
5030,0.000000,0.0,74.983217,0.000000
12239,0.000000,0.0,0.000000,79.059407
14466,0.000000,0.0,67.003603,0.000000
15402,79.916249,0.0,0.000000,0.000000
15704,0.000000,0.0,0.000000,79.593182
16027,0.000000,0.0,37.434842,37.434842
16544,0.000000,0.0,54.991714,0.000000
23566,0.000000,0.0,0.000000,0.000000
29303,0.000000,0.0,0.000000,75.778809
30641,0.000000,0.0,0.000000,0.000000


merging buildings to jobs
merging buildings with jobs to parcels
merging buildings to parcels
merging households to buildings and parcels table
merging households to buildings
merging persons to households and buildings
hh build table has  416 rows
Removed 496 rows because they contain missing values


,node_access_nhw,node_access_nhb,node_access_nha,node_access_h
5030,0.000000,0.000000,65.954661,0.000000
12239,68.834946,0.000000,0.000000,0.000000
14466,0.000000,0.000000,58.641230,0.000000
15402,0.000000,0.000000,69.926078,0.000000
15704,0.000000,0.000000,69.677733,0.000000
16027,0.000000,21.995446,0.000000,43.990891
16544,0.000000,0.000000,47.434394,0.000000
23566,0.000000,0.000000,0.000000,0.000000
30641,0.000000,0.000000,0.000000,0.000000
36421,0.000000,0.000000,58.152942,0.000000
